---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [171]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    state = ""
    RegionName = ""
    data = []
    with open('university_towns.txt') as f:
        content = f.readlines()
    for line in content:
        if "[edit]" in line:  # Starting new state
            state = line.split("[")[0]
        else: # extract Region Name
            RegionName = line.split("(")[0].strip()
            data.append([state,RegionName])
        
    result = pd.DataFrame(data) 
    result.columns = ['State','RegionName']
    return result
   

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [420]:
#     A recession is defined as starting with two consecutive quarters of GDP decline, 
#     and ending with two consecutive quarters of GDP growth.

def find_recession(ds):
    prevGDP = 0
    QtBeforeRecession = ''
    RecessionStartQt = ''
    RecessionBottomQt = ''
    lowestGDP = -1
    decline = 0
    growths = 0
    for iter_index,row in ds.iterrows():
        if prevGDP < row['GDP']:
            if (decline >= 2): # we are going up while in a recession!
                if growth == 0: # starting to grow
                    growth = 1
                else: #we are out of recession, two consecutive growths
                    return [RecessionStartQt,RecessionBottomQt,row['Quarter']]
            else:   # recession hasnt started so we reset
                decline = 0
                growth = 0
                RecessionStartQt = ''
                RecessionBottomQt = ''
                lowestGDP = -1
        else: # recession time?
            if decline == 0: #first decline lets save it
                RecessionStartQt = row['Quarter']
                lowestGDP = row['GDP']
                RecessionBottomQt = RecessionStartQt
            else: 
                if lowestGDP > row['GDP']:
                    lowestGDP = row['GDP']
                    RecessionBottomQt = row['Quarter']
            decline += 1
            growth = 0
        prevGDP = row['GDP']
    return ['','',''] # in case no recession was found
        

In [436]:
# %%time

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # using 2009 dollars and 2000 forward
    gdp = pd.read_excel('gdplev.xls',skiprows=219,usecols=[4,6],names=["Quarter","GDP"])
    return find_recession(gdp)[0]

get_recession_start()



'2008q3'

In [368]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=219,usecols=[4,6],names=["Quarter","GDP"])
    return find_recession(gdp)[2]

get_recession_end()

'2009q4'

In [369]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=219,usecols=[4,6],names=["Quarter","GDP"])
    return find_recession(gdp)[1]
    
get_recession_bottom()

'2009q2'

In [370]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing = pd.read_csv("City_Zhvi_AllHomes.csv")
    # concat since group by cant handle multiple non unique index
    housing['State'] = housing['State'].apply(lambda x: states[x] + "-") + housing['RegionName'] 
    housing = housing.drop(['RegionID', 'Metro','CountyName','SizeRank','RegionName'], axis=1)
    housing = housing.set_index(['State'])
    #Transpose to set rows in terms of quarters
    housing = housing.T
    housing['Quarter'] = housing.apply(get_Quarter,axis=1)
    housing  = housing.groupby(['Quarter']).mean()
    housing = housing[housing.index >= '2000q1']
    #Transpose to go back to our original index: State and RegionName
    housing = housing.T
    housing = housing.reset_index()
    housing['RegionName'] = housing['State'].apply(lambda x: x.split("-")[1])
    housing['State'] = housing['State'].apply(lambda x: x.split("-")[0])
    housing = housing.set_index(['State','RegionName'])
    return housing


def get_Quarter(row):
    if "-" not in row.name:
        return row.name
    else:
        qts = {'01':'q1','02':'q1','03':'q1',
               '04':'q2','05':'q2','06':'q2',
               '07':'q3','08':'q3','09':'q3',
               '10':'q4','11':'q4','12':'q4'}
        result = row.name.split("-")
        return result[0] + qts[result[1]]
        
    
convert_housing_data_to_quarters().shape
# 10730 rows × 67 columns

(10730, 67)

In [501]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing_qt = convert_housing_data_to_quarters()
    # get qt before recession start and recession bottom
    qt_before_rec = get_prev_quarter(get_recession_start())
    qt_rec_bottom = get_recession_bottom()    
    
    #calculate price ratio
    #price_ratio=quarter_before_recession/recession_bottom
    housing_qt["price_ratio"] = housing_qt[qt_before_rec]/housing_qt[qt_rec_bottom]
    housing_qt = housing_qt[['price_ratio']].dropna()
    
#     # create two df, uni_town_vals and n_uni_town_vals
    uni_towns = get_list_of_university_towns()
    housing_qt = housing_qt.merge(uni_towns,left_index=True, right_on=['State','RegionName'],indicator=True, how='left')
    uni_town_vals = housing_qt[housing_qt['_merge']=='both'].drop('_merge',axis=1).dropna()
    n_uni_town_vals = housing_qt[housing_qt['_merge']=='left_only'].drop('_merge',axis=1).dropna()
    
#     Hypothesis: University towns have their mean housing prices less effected by recessions.
#     null hypothesis: University towns and not university towns are affected equally during recession
   
    # t test tells you how significant the differences between groups are; 
    # In other words it lets you know if those differences (measured in means/averages) could have happened by chance.
    # Hi T shows how different two datasets are, P shows the probability that the result was due to chance
    ttest = ttest_ind(uni_town_vals['price_ratio'], n_uni_town_vals['price_ratio'])
    different = False
    better = 'university town'
    if ttest[1] < 0.01:
        #null hypothesis rejected
        #  small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, 
        # so you reject the null hypothesis.
        
        # A large p-value (> 0.05) indicates weak evidence against the null hypothesis, 
        # so it would fail to reject the null hypothesis.
        different = True

    if ttest[0] > 0:  # a positive T value: mean of n_uni_towns is smaller than uni_towns
        better = 'non-university town'

    return [different, ttest[1], better]
        

def get_prev_quarter(cur_qt): 
    qts = cur_qt.split("q")
    year = cur_qt[0]
    qt = cur_qt[1]
    
    if qts[1] == '1':
        return str(int(qts[0])-1) + 'q4'
    else:
        return qts[0] + 'q' + str(int(qts[1])-1)
    

run_ttest()


[True, 0.0030867070793210018, 'university town']